In [1]:
#节点及一阶邻居构成的局部图
import torch
import copy
import numpy as np
#from torch.utils.data import Dataset
import argparse
import time
import numpy
from pygod.utils import load_data
from pygod.generator import gen_contextual_outlier, gen_structural_outlier
from sklearn.model_selection import train_test_split
from torch_geometric.utils import add_self_loops, to_dense_adj, to_scipy_sparse_matrix, k_hop_subgraph
from dataset_process.local_graph_spectral import local_graph, neighbor_pad_or_truncate, gen_joint_structural_outlier
from sklearn.metrics import f1_score, accuracy_score, recall_score, roc_auc_score, precision_score, confusion_matrix
from torch_geometric.data import Data
from pygod.utils.utility import check_parameter
from scipy.sparse import csgraph
from scipy.sparse.linalg import eigsh
from model.High_pass_GCN import *

In [2]:
# 1. 初始化超参数
parser = argparse.ArgumentParser(description='parameters')
parser.add_argument('--dataset', type=str, default="disney") #"weibo"，"inj_cora"，"enron","disney","books","reddit"
#是否 normalize features
parser.add_argument('--normalize_feat', type=bool, default=True)
parser.add_argument('--calculate_contextual', type=bool, default=True)
parser.add_argument('--contextual_n', type=int, default=3)
parser.add_argument('--contextual_k', type=int, default=5)
parser.add_argument('--calculate_structural', type=bool, default=True)
parser.add_argument('--structural_n', type=int, default=3)
parser.add_argument('--structural_m', type=int, default=5)
parser.add_argument('--use_combine_outlier', type=bool, default=False)

#contextual_n =183  #weibo: 434 # books 14 # cora: 70；dinesy: 3; reddit: 183; Enron: 3
#contextual_k =30  #weibo: 10 # books 5 # Cora: 10；dinesy: 5; reddit: 30; Enron: 25
#structural_n =183  #weibo: 434 # books 14 # Cora: 70；dinesy: 3; reddit: 183; Enron: 3,
#structural_m =30  #weibo: 10 # books 5 # Cora: 10；dinesy: 5; reddit: 30; Enron: 25,

#reddit: 64+32; disney:28+32; weibo: 400+32, enron: 18+32; books:21+32; inj_cora:1433+32  ;
parser.add_argument("--hid_dim_c", type=int, default=28, help="Hidden layer dimension") 
parser.add_argument("--hid_dim_s", type=int, default=32, help="Hidden layer dimension") 
parser.add_argument("--hid_dim_j", type=int, default=32, help="Hidden layer dimension") 

parser.add_argument("--train_ratio", type=float, default=0.4, help="Training ratio")
parser.add_argument("--epoch", type=int, default=500, help="The max number of epochs") # For yelp，epoch =200; For amazon/tfinance/tsocial，epoch =100
parser.add_argument("--run", type=int, default=1, help="Running times")
parser.add_argument("--k_c", type=int, default=2, help="k_c in ChebConv")
parser.add_argument("--k_s", type=int, default=2, help="k_s in ChebConv")
parser.add_argument("--k_j", type=int, default=2, help="k_j in ChebConv")
from torch_geometric.utils import to_scipy_sparse_matrix
from scipy.sparse.linalg import eigs

args = parser.parse_args(args = [])

#2. 读取数据集
dataset_str = args.dataset
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
data = load_data(dataset_str)

# 计算最大特征值
edge_index = data.edge_index
num_nodes = data.num_nodes  # 如果需要提供节点数量
edge_index_tensor = to_scipy_sparse_matrix(edge_index, num_nodes=num_nodes)

In [3]:
#2. 节点特征归一化
node_features = data.x 
if args.normalize_feat:
    #特征归一化
    node_features_min = node_features.min()
    node_features_max = node_features.max()
    node_features = (node_features - node_features_min)/node_features_max
    data.x = node_features

In [4]:
# 3.产生结构、属性和joint anomaly
calculate_contextual=args.calculate_contextual
calculate_structural=args.calculate_structural
yc = []
ys = []
yj = []
    
if calculate_contextual:
        
    if dataset_str == "inj_cora":
        yc = data.y >> 0 & 1 # contextual outliers
    else:
        data, yc = gen_contextual_outlier(data=data,n=args.contextual_n,k=args.contextual_k)
            
    yc = yc.cpu().detach()
    
    
if calculate_structural:
        
    if dataset_str == "inj_cora":
        ys = data.y >> 1 & 1 # structural outliers
    else:
        data, ys = gen_structural_outlier(data=data,n=args.structural_n,m=args.structural_m,p=0.2)
            
    data, yj = gen_joint_structural_outlier(data=data,n=args.structural_n,m=args.structural_m)
        
    
if args.use_combine_outlier:
    data.y = torch.logical_or(ys, yc).int()
        
ysj = torch.logical_or(ys, yj).int()
data.y = data.y.bool()    # binary labels (inlier/outlier)

ys = ys.bool()
yj = yj.bool()

#添加自环
data.edge_index, _ = add_self_loops(data.edge_index)

In [5]:
# 4.计算local graph features 并归一化
def compute_padded_laplacian_eigenvalues(data, node_idx, k=1, max_len=32):
    """
    计算 `torch_geometric.data.Data` 中某个节点的局部拉普拉斯特征值（归一化），并补零至 max_len 维度。
    参数:
    - data: `torch_geometric.data.Data` 图数据对象
    - node_idx: 目标节点索引
    - k: k-hop 邻居范围
    - max_len: 期望的特征向量长度（不足补零）
    返回:
    - eigvals: 长度为 max_len 的特征值向量
    """
    # a. 获取 k-hop 局部子图
    subset, edge_index, _, _ = k_hop_subgraph(node_idx, k, data.edge_index, relabel_nodes=True)
    
    # b. 转换为 scipy 稀疏矩阵
    A = to_scipy_sparse_matrix(edge_index, num_nodes=len(subset))
    
    # c. 计算归一化拉普拉斯矩阵
    L_sym = csgraph.laplacian(A, normed=True)
    
    # d. 计算前 max_len 个最小特征值
    num_eigenvalues = min(A.shape[0] - 1, max_len)  # 避免超出子图大小
    
    if(num_eigenvalues>0):
        eigvals, _ = eigsh(L_sym, k=num_eigenvalues, which='SM')
    else:
        eigvals = np.zeros(2)

    # 5. 补零至 max_len
    eigvals = np.pad(eigvals, (0, max_len - len(eigvals)), 'constant', constant_values=0)
    if args.normalize_feat:
        #特征归一化
        neighbor_spectral_features_min = eigvals.real.min()
        neighbor_spectral_featuress_max = eigvals.real.max()
        if ((neighbor_spectral_featuress_max-1e-4))>0:
            neighbor_spectral_features = (eigvals.real - neighbor_spectral_features_min)/neighbor_spectral_featuress_max
            return torch.tensor(neighbor_spectral_features, dtype=torch.float32)
            
        else:
            return torch.tensor(eigvals, dtype=torch.float32)

sorted_tensors = []
for i in range(data.x.shape[0]):
    eigvals_padded = compute_padded_laplacian_eigenvalues(data, node_idx=0, k=1, max_len=32)
    sorted_tensors.append(eigvals_padded)

# 将所有一维 Tensor 堆叠成一个二维 Tensor（每个一维 Tensor 作为一行）
neighbor_spectral_features = torch.stack(sorted_tensors, dim=0)                

In [6]:
#5. 将local grpah spectrum编码入图节点特征
#neighbor_spectral_features = torch.cat((node_features, neighbor_spectral_features), dim=1)

In [7]:
def train(model_c, model_s, model_j, data, neighbor_spectral_features,ys, yj, args):
    # contextual_outlier
    features_c = data.x
    labels_c = data.y
    features_s = neighbor_spectral_features
    features_j = neighbor_spectral_features
    labels_s = ys
    labels_j = yj
    index = list(range(len(labels_c)))

    idx_train_c, idx_rest_c, y_train_c, y_rest_c = train_test_split(index, labels_c[index], stratify=labels_c[index],
                                                            train_size=args.train_ratio,
                                                            random_state=2, shuffle=True)
    idx_valid_c, idx_test_c, y_valid_c, y_test_c = train_test_split(idx_rest_c, y_rest_c, stratify=y_rest_c,
                                                            test_size=0.67,
                                                            random_state=2, shuffle=True)
    idx_train_s, idx_rest_s, y_train_s, y_rest_s = train_test_split(index, labels_s[index], stratify=labels_s[index],
                                                            train_size=args.train_ratio,
                                                            random_state=2, shuffle=True)
    idx_valid_s, idx_test_s, y_valid_s, y_tes_s = train_test_split(idx_rest_s, y_rest_s, stratify=y_rest_s,
                                                            test_size=0.67,
                                                            random_state=2, shuffle=True)
    idx_train_j, idx_rest_j, y_train_j, y_rest_j = train_test_split(index, labels_j[index], stratify=labels_j[index],
                                                            train_size=args.train_ratio,
                                                            random_state=2, shuffle=True)
    idx_valid_j, idx_test_j, y_valid_j, y_tes_j = train_test_split(idx_rest_j, y_rest_j, stratify=y_rest_j,
                                                            test_size=0.67,
                                                            random_state=2, shuffle=True)
    train_mask_c = torch.zeros([len(labels_c)]).bool()
    val_mask_c = torch.zeros([len(labels_c)]).bool()
    test_mask_c = torch.zeros([len(labels_c)]).bool()
    
    train_mask_s = torch.zeros([len(labels_s)]).bool()
    val_mask_s = torch.zeros([len(labels_s)]).bool()
    test_mask_s = torch.zeros([len(labels_s)]).bool()
    
    train_mask_j = torch.zeros([len(labels_j)]).bool()
    val_mask_j = torch.zeros([len(labels_j)]).bool()
    test_mask_j = torch.zeros([len(labels_j)]).bool()
    
    print(" y_train:", y_train_c.shape, sum(y_train_c))
    
    train_mask_c[idx_train_c] = 1
    val_mask_c[idx_valid_c] = 1
    test_mask_c[idx_test_c] = 1
    
    train_mask_s[idx_train_s] = 1
    val_mask_s[idx_valid_s] = 1
    test_mask_s[idx_test_s] = 1
    
    train_mask_j[idx_train_j] = 1
    val_mask_j[idx_valid_j] = 1
    test_mask_j[idx_test_j] = 1
    print('train/dev/test samples c: ', train_mask_c.sum().item(), val_mask_c.sum().item(), test_mask_c.sum().item())
    print('train/dev/test samples  s: ', train_mask_s.sum().item(), val_mask_s.sum().item(), test_mask_s.sum().item())
    print('train/dev/test samples  j: ', train_mask_j.sum().item(), val_mask_j.sum().item(), test_mask_j.sum().item())
    
    optimizer_c = torch.optim.Adam(model_c.parameters(), lr=0.01)
    optimizer_s = torch.optim.Adam(model_s.parameters(), lr=0.01)
    optimizer_j = torch.optim.Adam(model_j.parameters(), lr=0.01)
    
    best_f1_c, final_tf1_c, final_trec_c, final_tpre_c, final_tmf1_c, final_tauc_c = 0., 0., 0., 0., 0., 0.
    best_f1_s, final_tf1_s, final_trec_s, final_tpre_s, final_tmf1_s, final_tauc_s = 0., 0., 0., 0., 0., 0.
    best_f1_j, final_tf1_j, final_trec_j, final_tpre_j, final_tmf1_j, final_tauc_j = 0., 0., 0., 0., 0., 0.

    #对bool型的不允许“-”操作1-labels[train_mask] 改为：~labels[train_mask]
    weight_c = (~labels_c[train_mask_c]).sum().item() / labels_c[train_mask_c].sum().item()
    weight_s = (~labels_s[train_mask_s]).sum().item() / labels_s[train_mask_s].sum().item()
    weight_j = (~labels_j[train_mask_j]).sum().item() / labels_j[train_mask_j].sum().item()
    print('cross entropy weight: ', weight_c, weight_s, weight_j)
    time_start = time.time()
    for e in range(args.epoch):
        # 训练
        model_c.train()
        model_s.train()
        model_j.train()
        
        # 调用模型中的forward函数
        logits_c = model_c(features_c,data)
        logits_s = model_s(features_s,data)
        logits_j = model_j(features_j,data)
        
        #labels[train_mask] 需要long型
        loss_c = F.cross_entropy(logits_c[train_mask_c], labels_c[train_mask_c].to(torch.long), weight = torch.tensor([1., weight_c]))
        loss_s = F.cross_entropy(logits_s[train_mask_s], labels_s[train_mask_s].to(torch.long), weight = torch.tensor([1., weight_s]))
        loss_j = F.cross_entropy(logits_j[train_mask_j], labels_j[train_mask_j].to(torch.long), weight = torch.tensor([1., weight_j]))
        
        optimizer_c.zero_grad()
        optimizer_s.zero_grad()
        optimizer_j.zero_grad()
        
        loss_c.backward()
        loss_s.backward()
        loss_j.backward()
        
        optimizer_c.step()
        optimizer_s.step()
        optimizer_j.step()
        
        #验证
        model_c.eval()
        model_s.eval()
        model_j.eval()
        
        probs_c = logits_c.softmax(1)
        probs_s = logits_s.softmax(1)
        probs_j = logits_j.softmax(1)
        
        f1_c, thres_c = get_best_f1(labels_c[val_mask_c], probs_c[val_mask_c])
        f1_s, thres_s = get_best_f1(labels_s[val_mask_s], probs_s[val_mask_s])
        f1_j, thres_j = get_best_f1(labels_j[val_mask_j], probs_j[val_mask_j])
        
        preds_c = numpy.zeros_like(labels_c)
        preds_c[probs_c[:, 1] > thres_c] = 1
        preds_s = numpy.zeros_like(labels_s)
        preds_s[probs_s[:, 1] > thres_s] = 1
        preds_j = numpy.zeros_like(labels_j)
        preds_j[probs_j[:, 1] > thres_j] = 1

        trec_c = recall_score(labels_c[test_mask_c], preds_c[test_mask_c])
        tpre_c = precision_score(labels_c[test_mask_c], preds_c[test_mask_c])
        tmf1_c = f1_score(labels_c[test_mask_c], preds_c[test_mask_c], average='macro')
        tauc_c = roc_auc_score(labels_c[test_mask_c], probs_c[test_mask_c][:, 1].detach().numpy())
        
        trec_s = recall_score(labels_s[test_mask_s], preds_s[test_mask_s])
        tpre_s = precision_score(labels_s[test_mask_s], preds_s[test_mask_s])
        tmf1_s = f1_score(labels_s[test_mask_s], preds_s[test_mask_s], average='macro')
        tauc_s = roc_auc_score(labels_s[test_mask_s], probs_s[test_mask_s][:, 1].detach().numpy())

        trec_j = recall_score(labels_j[test_mask_j], preds_j[test_mask_j])
        tpre_j = precision_score(labels_j[test_mask_j], preds_j[test_mask_j])
        tmf1_j = f1_score(labels_j[test_mask_j], preds_j[test_mask_j], average='macro')
        tauc_j = roc_auc_score(labels_j[test_mask_j], probs_j[test_mask_j][:, 1].detach().numpy())

        if best_f1_c < f1_c:
            best_f1_c = f1_c
            final_trec_c = trec_c
            final_tpre_c = tpre_c
            final_tmf1_c = tmf1_c
            final_tauc_c = tauc_c
        print('C: Epoch {}, loss: {:.4f}, val mf1: {:.4f}, (best {:.4f})'.format(e, loss_c, f1_c, best_f1_c))

        if best_f1_s < f1_s:
            best_f1_s = f1_s
            final_trec_s = trec_s
            final_tpre_s = tpre_s
            final_tmf1_s = tmf1_s
            final_tauc_s = tauc_s
        print('s: Epoch {}, loss: {:.4f}, val mf1: {:.4f}, (best {:.4f})'.format(e, loss_s, f1_s, best_f1_s))


        if best_f1_j < f1_j:
            best_f1_j = f1_j
            final_trec_j = trec_j
            final_tpre_j = tpre_j
            final_tmf1_j = tmf1_j
            final_tauc_j = tauc_j
        print('J: Epoch {}, loss: {:.4f}, val mf1: {:.4f}, (best {:.4f})'.format(e, loss_j, f1_j, best_f1_j))

    time_end = time.time()
    print('time cost: ', time_end - time_start, 's')
    print('Test_c: REC {:.2f} PRE {:.2f} MF1 {:.2f} AUC {:.2f}'.format(final_trec_c*100,
                                                                     final_tpre_c*100, final_tmf1_c*100, final_tauc_c*100))
    print('Test_s: REC {:.2f} PRE {:.2f} MF1 {:.2f} AUC {:.2f}'.format(final_trec_s*100,
                                                                     final_tpre_s*100, final_tmf1_s*100, final_tauc_s*100))
    print('Test_j: REC {:.2f} PRE {:.2f} MF1 {:.2f} AUC {:.2f}'.format(final_trec_j*100,
                                                                     final_tpre_j*100, final_tmf1_j*100, final_tauc_j*100))

    return final_tmf1_c, final_tauc_c, final_tmf1_s, final_tauc_s, final_tmf1_j, final_tauc_j


# threshold adjusting for best macro f1
def get_best_f1(labels, probs):
    best_f1, best_thre = 0, 0
    for thres in np.linspace(0.05, 0.95, 19):
        #构建一个与labels同维度的数组,并初始化所有变量为零
        preds = np.zeros_like(labels)
        preds[probs[:,1] > thres] = 1
        #average='binary'：计算二分类问题中的 F1 分数（默认值）。
        #average='micro'：对所有类别的真实和预测样本进行汇总，然后计算 F1 分数。
        #average='macro'：计算每个类别的 F1 分数，然后取平均值。
        #average=None：返回每个类别的 F1 分数。
        # F1_score 详细原理间“备份”
        mf1 = f1_score(labels, preds, average='macro')
        if mf1 > best_f1:
            best_f1 = mf1
            best_thre = thres
    return best_f1, best_thre


In [8]:
h_feats_c = args.hid_dim_c
in_feats_c = data.x.shape[1]

h_feats_s = args.hid_dim_s
in_feats_s = neighbor_spectral_features.shape[1]

h_feats_j = args.hid_dim_j
in_feats_j = neighbor_spectral_features.shape[1]
num_classes = 2

if args.run == 0:
    model_c = ChebConvGAD_c(in_feats_c, h_feats_c, num_classes, k = args.k_c)
    model_s = ChebConvGAD_s(in_feats_s, h_feats_s, num_classes, k = args.k_s)
    model_j = ChebConvGAD_j(in_feats_j, h_feats_j, num_classes, k = args.k_j)
    
    train(model_c, model_s, model_j, data, neighbor_spectral_features, ys, yj, args)

else:
    final_mf1s_c, final_aucs_c, final_mf1s_s, final_aucs_s,final_mf1s_j, final_aucs_j = [], [], [], [], [], []
    for tt in range(args.run):

        #in_feats 特征点维度；h_feats：隐层维度；num_classes：节点分类数（nomal，anomaly）
        model_c = ChebConvGAD_c(in_feats_c, h_feats_c, num_classes, k = args.k_c)
        model_s = ChebConvGAD_s(in_feats_s, h_feats_s, num_classes, k = args.k_s)
        model_j = ChebConvGAD_j(in_feats_j, h_feats_j, num_classes, k = args.k_j)
        mf1_c, auc_c, mf1_s, auc_s,mf1_j, auc_j = train(model_c, model_s, model_j, data, neighbor_spectral_features, ys, yj, args)
        
        final_mf1s_c.append(mf1_c)
        final_aucs_c.append(auc_c)
        final_mf1s_s.append(mf1_s)
        final_aucs_s.append(auc_s)
        final_mf1s_j.append(mf1_j)
        final_aucs_j.append(auc_j)
        
    final_mf1s_c = np.array(final_mf1s_c)
    final_aucs_c = np.array(final_aucs_c)
    # np.std :计算全局标准差
    print('MF1-mean_c: {:.2f}, MF1-std: {:.2f}, AUC-mean: {:.2f}, AUC-std: {:.2f}'.format(100 * np.mean(final_mf1s_c),
                                                                                            100 * np.std(final_mf1s_c),
                                                               100 * np.mean(final_aucs_c), 100 * np.std(final_aucs_c)))

    final_mf1s_s = np.array(final_mf1s_s)
    final_aucs_s = np.array(final_aucs_s)
    # np.std :计算全局标准差
    print('MF1-mean_c: {:.2f}, MF1-std: {:.2f}, AUC-mean: {:.2f}, AUC-std: {:.2f}'.format(100 * np.mean(final_mf1s_s),
                                                                                            100 * np.std(final_mf1s_s),
                                                               100 * np.mean(final_aucs_s), 100 * np.std(final_aucs_s)))

    final_mf1s_j = np.array(final_mf1s_j)
    final_aucs_j = np.array(final_aucs_j)
    # np.std :计算全局标准差
    print('MF1-mean_j: {:.2f}, MF1-std: {:.2f}, AUC-mean: {:.2f}, AUC-std: {:.2f}'.format(100 * np.mean(final_mf1s_j),
                                                                                            100 * np.std(final_mf1s_j),
                                                               100 * np.mean(final_aucs_j), 100 * np.std(final_aucs_j)))

K.....: 2
K.....: 2
K.....: 2
 y_train: torch.Size([49]) tensor(2)
train/dev/test samples c:  49 24 51
train/dev/test samples  s:  49 24 51
train/dev/test samples  j:  49 24 51
cross entropy weight:  23.5 7.166666666666667 48.0
C: Epoch 0, loss: 0.6960, val mf1: 0.4894, (best 0.4894)
s: Epoch 0, loss: 0.6934, val mf1: 0.4667, (best 0.4667)
J: Epoch 0, loss: 0.6934, val mf1: 0.4894, (best 0.4894)


C:\ProgramData\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\ProgramData\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\ProgramData\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\ProgramData\anaconda3\lib\site-packages\sklearn\metrics\_clas

C: Epoch 1, loss: 0.6781, val mf1: 0.4894, (best 0.4894)
s: Epoch 1, loss: 0.6935, val mf1: 0.4667, (best 0.4667)
J: Epoch 1, loss: 0.6935, val mf1: 0.4894, (best 0.4894)
C: Epoch 2, loss: 0.6505, val mf1: 0.4894, (best 0.4894)
s: Epoch 2, loss: 0.6933, val mf1: 0.4667, (best 0.4667)
J: Epoch 2, loss: 0.6933, val mf1: 0.4894, (best 0.4894)


C:\ProgramData\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\ProgramData\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\ProgramData\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\ProgramData\anaconda3\lib\site-packages\sklearn\metrics\_clas

C: Epoch 3, loss: 0.6137, val mf1: 0.4894, (best 0.4894)
s: Epoch 3, loss: 0.6932, val mf1: 0.4667, (best 0.4667)
J: Epoch 3, loss: 0.6932, val mf1: 0.4894, (best 0.4894)
C: Epoch 4, loss: 0.5701, val mf1: 0.4894, (best 0.4894)
s: Epoch 4, loss: 0.6933, val mf1: 0.4667, (best 0.4667)
J: Epoch 4, loss: 0.6933, val mf1: 0.4894, (best 0.4894)
C: Epoch 5, loss: 0.5228, val mf1: 0.4894, (best 0.4894)
s: Epoch 5, loss: 0.6932, val mf1: 0.4667, (best 0.4667)
J: Epoch 5, loss: 0.6932, val mf1: 0.4894, (best 0.4894)


...

C:\ProgramData\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\ProgramData\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\ProgramData\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\ProgramData\anaconda3\lib\site-packages\sklearn\metrics\_clas

C: Epoch 495, loss: 0.0000, val mf1: 0.4783, (best 0.7273)
s: Epoch 495, loss: 0.6931, val mf1: 0.4667, (best 0.4667)
J: Epoch 495, loss: 0.6931, val mf1: 0.4894, (best 0.4894)
C: Epoch 496, loss: 0.0000, val mf1: 0.4783, (best 0.7273)
s: Epoch 496, loss: 0.6931, val mf1: 0.4667, (best 0.4667)
J: Epoch 496, loss: 0.6931, val mf1: 0.4894, (best 0.4894)


C:\ProgramData\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\ProgramData\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\ProgramData\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\ProgramData\anaconda3\lib\site-packages\sklearn\metrics\_clas

C: Epoch 497, loss: 0.0000, val mf1: 0.4783, (best 0.7273)
s: Epoch 497, loss: 0.6931, val mf1: 0.4667, (best 0.4667)
J: Epoch 497, loss: 0.6931, val mf1: 0.4894, (best 0.4894)
C: Epoch 498, loss: 0.0000, val mf1: 0.4783, (best 0.7273)
s: Epoch 498, loss: 0.6931, val mf1: 0.4667, (best 0.4667)
J: Epoch 498, loss: 0.6931, val mf1: 0.4894, (best 0.4894)
C: Epoch 499, loss: 0.0000, val mf1: 0.4783, (best 0.7273)
s: Epoch 499, loss: 0.6931, val mf1: 0.4667, (best 0.4667)
J: Epoch 499, loss: 0.6931, val mf1: 0.4894, (best 0.4894)
time cost:  54.03340530395508 s
Test_c: REC 33.33 PRE 8.33 MF1 49.20 AUC 74.31
Test_s: REC 0.00 PRE 0.00 MF1 46.88 AUC 50.00
Test_j: REC 0.00 PRE 0.00 MF1 49.50 AUC 50.00
MF1-mean_c: 49.20, MF1-std: 0.00, AUC-mean: 74.31, AUC-std: 0.00
MF1-mean_c: 46.88, MF1-std: 0.00, AUC-mean: 50.00, AUC-std: 0.00
MF1-mean_j: 49.50, MF1-std: 0.00, AUC-mean: 50.00, AUC-std: 0.00


C:\ProgramData\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\ProgramData\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\ProgramData\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\ProgramData\anaconda3\lib\site-packages\sklearn\metrics\_clas